In [1]:
import numpy as np
import pandas as pd
from random import randrange
from scipy.stats.stats import pearsonr
from sklearn.decomposition import PCA
import math
from scipy.stats import ttest_ind, ttest_ind_from_stats

data = pd.read_csv('../Research Project/namesR.csv', low_memory=False)


/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
users = pd.Series(data.user).unique()
items = pd.Series(data.item).unique()
items = np.sort(items)

openness = []
conscientiousness = []
extraversion = []
agreeableness = []
neuroticism = []

for user in users:
    dfuser = data.loc[data['user'] == user]
    openness.append(dfuser.iloc[0]['openness'])
    conscientiousness.append(dfuser.iloc[0]['conscientiousness'])
    extraversion.append(dfuser.iloc[0]['extraversion'])
    agreeableness.append(dfuser.iloc[0]['agreeableness'])
    neuroticism.append(dfuser.iloc[0]['neuroticism'])

df_user_personality = pd.DataFrame({ 'user' : users,
                                     'openness' : openness,
                                     'conscientiousness' : conscientiousness,
                                     'extraversion' : extraversion,
                                     'agreeableness' : agreeableness,
                                     'neuroticism' : neuroticism })


df_user_personality.head()
col_names =  list(data)
users = df_user_personality.index + 1
sample0 = pd.DataFrame(columns = col_names)
sample1 = pd.DataFrame(columns = col_names)
sample2 = pd.DataFrame(columns = col_names)
sample3 = pd.DataFrame(columns = col_names)
sample4 = pd.DataFrame(columns = col_names)

for user in users:
    df_user = data.loc[data['user'] == user]
    s0 = np.array_split(df_user, 5)[0]
    s1 = np.array_split(df_user, 5)[1]
    s2 = np.array_split(df_user, 5)[2]
    s3 = np.array_split(df_user, 5)[3]
    s4 = np.array_split(df_user, 5)[4]
    sample0 = pd.concat([sample0, s0])
    sample1 = pd.concat([sample1, s1])
    sample2 = pd.concat([sample2, s2])
    sample3 = pd.concat([sample3, s3])
    sample4 = pd.concat([sample4, s4])

In [23]:
def matrix_factorization_noP(R, P, Q, K, rating_mean, user_bias, item_bias, steps=5000, alpha=0.0002, beta=0.02):
    Q = Q.T
    for step in range(steps):
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    eij = R[i][j] - (rating_mean + item_bias[j] + user_bias[i] + np.dot(P[i,:],Q[:,j]))
                    item_bias[j] = item_bias[j] + alpha * (eij - beta * item_bias[j]) 
                    user_bias[i] = user_bias[i] + alpha * (eij - beta * user_bias[i]) 
                    for k in range(K):
                        P[i][k] = P[i][k] + alpha * (eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (eij * P[i][k] - beta * Q[k][j])
    return P, Q.T

def metrics(rated_items_test, predictions_noP):
    
    precisionV = 0
    totalP = 0

    recallV = 0
    totalR = 0
    
    ndcgV = 0
    totalN = len(users)

    nrec = 10

    for user in users:

        itemsR = rated_items_test.T.nlargest(nrec, user)[0:10][user].index.tolist()
        itemsP = predictions_noP.T.nlargest(10, user)[0:10][user].index.tolist()
        
        score = []
        #print(ratingsP)
        for item in itemsP:
            meanrating = np.mean(predictions_noP[item])
            score.append(meanrating)
        #print(score)
        a = np.asfarray(score)
        b = np.sort(a)
        b = np.asfarray(b[::-1])
        #print(b)
        t =  a[0] + np.sum(a[1:] / np.log2(np.arange(2, a.size + 1)))
        t2 =  b[0] + np.sum(b[1:] / np.log2(np.arange(2, b.size + 1)))
        
        #print(t/t2)
        
        ndcgV = ndcgV + (t/t2)
        
        for i in range(len(itemsP)):
            itemsP[i] = int(itemsP[i]/100)
    
        totalP = totalP + len(itemsR)
    
        df = predictions_noP.copy().T
    
        ratingsR = rated_items_test.T.nlargest(nrec, user)[0:10][user].tolist()
        ratingsP = df.nlargest(10, user)[0:10][user].tolist()
    
        for item in itemsR:
            if item in itemsP:
                precisionV = precisionV + 1
            if(ratingsR[itemsR.index(item)]>3):
                totalR = totalR + 1
                if(item in itemsP):
                    if(ratingsP [itemsP.index(item)]>3.5):
                        recallV = recallV+1 
                        
    T = np.array(rated_items_test.astype(float))
    R = np.array(predictions_noP.astype(float))                    
    error = 0
    totalE = 0
    for i in range(len(T)):
        for j in range(len(T[i])):
            if(T[i][j]>0):
                totalE = totalE + 1
                sqe = (T[i][j] - R[i][j])
                error = sqe*sqe
    
    return (precisionV/totalP)*100, (recallV/totalR)*100, math.sqrt(error/totalE), ndcgV/totalN

In [24]:
precision = np.empty([1,5])
recall = np.empty([1,5])
ndcg = np.empty([1,5])
error = np.empty([1,5])

for iteration in range(5):
    print('ITERATION')
    print(iteration)
    if(iteration == 0):
        train = pd.concat([sample0, sample1, sample2, sample3])
        test = sample4
    elif(iteration == 1):
        train = pd.concat([sample1, sample2, sample3, sample4])
        test = sample0
    elif(iteration == 2):
        train = pd.concat([sample2, sample3, sample4, sample0])
        test = sample1        
    elif(iteration == 3):
        train = pd.concat([sample3, sample4, sample0, sample1])
        test = sample2
    else:
        train = pd.concat([sample4, sample0, sample1, sample2])
        test = sample3
    itemsOG = train['item'].copy()

    items = pd.Series(data.item).unique()
    items = np.sort(items)
    Npersonalities = len(df_user_personality.columns)-1
    MaxV = max(df_user_personality.loc[:, df_user_personality.columns != 'user'].max())
    totalclasses = MaxV*Npersonalities
    personality = ['agreeableness','conscientiousness','extraversion','neuroticism','openness']

    for item in items:
        df_item = train.loc[train['item'] == item]
        ratings = df_item['rating']
        tValue = []
        pValue = []
        for pers in personality:
            for i in range(1, MaxV+1):
                #print(pers + ': ' + str(i))
                rating_T = []
                rating_F = []
                position = df_item.index
                j = 0
                for rating in ratings:
                    if(df_item[pers][position[j]] == i):
                        rating_T.append(rating)
                    else:
                        rating_F.append(rating)
                    j = j + 1
                #print('/////')
                #print(rating_T)
                #print(len(rating_T))
                #print(rating_F)
                #print(len(rating_F))
                #print('///')
                if (len(rating_T) > 1 and len(rating_F) > 1):
                    t, p = ttest_ind(rating_T, rating_F, equal_var=False)
                    #print(t)
                    #print(p)
                    tValue.append(t)
                    pValue.append(p)
        if(len(tValue) > 0 and len(pValue) > 0):            
            maxT = -1000
            indexMaxT = 0
            for i in range(len(pValue)):
                if (pValue[i]<0.05) & (tValue[i]>maxT):
                    maxT = tValue[i]
                    indexMaxT = i
        
        personalityI = int(round(indexMaxT/7+0.51))-1
        personalityV = indexMaxT+1-(personalityI)*7
        
        
        #print(personality[personalityI] + " " + str(personalityV))
        
        for indexIN in df_item.index:
            if(df_item[personality[personalityI]][indexIN] == personalityV):
                #print('Hello')
                train['item'][indexIN] = int(str(item) + "01")
            else:
                #print('Hi')
                train['item'][indexIN] = int(str(item) + "02")
    
        else:
            continue
    rated_items_train = pd.DataFrame(index=users, columns=np.sort(pd.Series(train.item).unique()), dtype='float')

    rated_items_test = pd.DataFrame(index=users, columns=items, dtype='float')

    for user in users:
        df_user_rat = train.loc[train['user'] == user]
        items_rat = df_user_rat['item'].tolist()
        rating = df_user_rat['rating'].tolist()
        
        df_user_rat_t = test.loc[test['user'] == user]
        items_rat_t = df_user_rat_t['item'].tolist()
        rating_t = df_user_rat_t['rating'].tolist()
        
        i = 0
        for item in items_rat:
            rated_items_train.loc[user][item] = rating[i]
            i = i + 1
        i = 0
        for item in items_rat_t:
            rated_items_test.loc[user][item] = rating_t[i]
            i = i + 1
    R = np.array(rated_items_train.astype(float))
    rating_mean = train['rating'].mean()
    N = len(R)
    M = len(R[0])
    K = 10
    P = np.random.normal(scale= 1/K, size=(N,K))
    Q = np.random.normal(scale= 1/K, size=(M,K))
    user_bias = np.random.normal(scale= 1/K, size=(N,1))
    item_bias = np.random.normal(scale= 1/K, size=(M,1))

    nP, nQ = matrix_factorization_noP(R, P, Q, K, rating_mean, user_bias, item_bias, steps = 120, alpha= 0.001, beta= 0.002)

    for i in range(len(R)):
        for j in range(len(R[i])):
            R[i][j] = rating_mean + user_bias[i] + item_bias[j] + np.dot(P[i,:],Q.T[:,j])
            
    predictions_noP = pd.DataFrame(index=users, columns=np.sort(pd.Series(train.item).unique()), data = R)
    p, r, e, n = metrics(rated_items_test, predictions_noP)
    #precision[0][iteration] = p
    #recall[0][iteration] = r
    precision[0][iteration] = p
    recall[0][iteration] = r
    ndcg[0][iteration] = ndcg
    error[0][iteration] = e
    print(p)
    print(r)
    print(e)
    print(n)

ITERATION
0
9.422492401215806
42.857142857142854
0.1374461003222707
0.999270818178852
ITERATION
1
16.666666666666664
16.627634660421545
0.028635271136281094
0.9995172228013469
ITERATION
2
16.05839416058394
24.034334763948497
0.08525999942018431
0.999010664345069
ITERATION
3
27.34584450402145
31.759656652360512
0.08433656918324954
0.999037439679321
ITERATION
4
20.689655172413794
30.21978021978022
0.06802775033360099
0.9991868463633051


## Some Metrics already recovered

In [18]:
precision_noP1 = [12.158054711246201, 19.017094017094017, 15.328467153284672, 20.37533512064343, 16.954022988505745]
recall_noP1 = [50.0, 20.140515222482435, 22.746781115879827, 25.75107296137339, 24.725274725274726]
e_noP1 = [0.12819255254417394, 0.03353121641931519, 0.08204314836723474, 0.07809071329681223, 0.06632819179168789]

precision_noP2 = [10.638297872340425, 18.162393162393162, 14.355231143552311, 21.179624664879356, 17.24137931034483]
recall_noP2 = [46.42857142857143, 6.0889929742388755, 19.742489270386265, 23.17596566523605, 22.52747252747253]
e_noP2 = [0.13518802561971324, 0.04296571741251353, 0.08172812043111354, 0.08507506175105764, 0.07559787901013024]

precision_noP3 = [19.45288753799392, 9.18803418803419, 12.895377128953772, 16.621983914209114, 14.655172413793101]
recall_noP3 = [0.0, 0.468384074941452, 13.304721030042918, 9.44206008583691, 16.483516483516482]
e_noP3 = [0.1551877742566284, 0.043044314275938536, 0.09305661886920405, 0.07178072152306228, 0.0734080471393372]

precision_noP4 = [10.94224924012158, 21.367521367521366, 13.62530413625304, 23.86058981233244, 18.39080459770115]
recall_noP4 = [50.0, 22.482435597189696, 18.4549356223176, 29.184549356223176, 28.57142857142857]
e_noP4 = [0.0748992849478179, 0.015957687848008884, 0.04948561647266341, 0.07065609635756304, 0.08695499217662961]

print('Alpha = 0.0001, Beta = 0.002, Nsteps = 100')
print(np.mean(precision_noP2))
print(np.mean(recall_noP2))
print(np.mean(e_noP2))
print()
print('Alpha = 0.00001, Beta = 0.002, Nsteps = 120')
print(np.mean(precision_noP3))
print(np.mean(recall_noP3))
print(np.mean(e_noP3))
print()
print('Alpha = 0.001, Beta = 0.02, Nsteps = 120')
print(np.mean(precision_noP1))
print(np.mean(recall_noP1))
print(np.mean(e_noP1))
print()
print('Alpha = 0.01, Beta = 0.02, Nsteps = 120')
print(np.mean(precision_noP4))
print(np.mean(recall_noP4))
print(np.mean(e_noP4))

Alpha = 0.0001, Beta = 0.002, Nsteps = 100
16.315385230702013
23.59269837318103
0.08411096084490563

Alpha = 0.00001, Beta = 0.002, Nsteps = 120
14.562691036596817
7.939736334867552
0.0872954952128341

Alpha = 0.001, Beta = 0.02, Nsteps = 120
16.766594798154813
28.672728805002073
0.07763716448384479

Alpha = 0.01, Beta = 0.02, Nsteps = 120
17.637293830785914
29.738669829431807
0.05959073556053657
